## ML notes from fast.ai course

Lesson source
http://course18.fast.ai/ml.html

Material source
https://forums.fast.ai/t/fastai-v0-7-install-issues-thread/24652

Data source
https://www.kaggle.com/c/bluebook-for-bulldozers/data

## Lesson 1
### Data import

Libraries

In [ ]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

Set the path to the data

In [ ]:
PATH = "data/bulldozers/"

In [ ]:
!ls {PATH}

<code>!</code> is for execute in bash, <code>{}</code> is to take variables from python

First step, **read the data with Pandas** (import always as pd)

In [ ]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

<code>print (f'{PATH}Train.csv')</code> is a usefull strategy to concatenate strings with variables (also numerical)

<code>parse_date=["column_name"]</code> is to specify dates column

If the dataset is very big **do not use** <code>low_memory=FALSE</code> but create a dictionary containig the data type of each column and pass it to <code>read_csv</code> as <code>dtype=dict</code> 

Efficient way to save temporarily the dataset: 

In [ ]:
os.makedirs('tmp', exist_ok=True)
df_raw.to_feather('tmp/bulldozers-raw')

And read it:

In [ ]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

To check the compuational time and the time used by each function, put the followings in front of the line of code

In [ ]:
%time or %prun

If the dataset is very large and you want to run RF many time is better to convert it in **np.array** before pass it to RF (RM does it automatically, but takes time)

In [ ]:
x = np.array(data, dtype=np.float32)

### Data pre-processing

Preliminary analysis and general information about the table, <code>.T</code> is to get the trasnsposed

In [ ]:
df_raw.columns

In [ ]:
df_raw.describe().T

In [ ]:
df_raw.column_name.head()

Use numpy (always import as np) to apply log to a given column

In [ ]:
df_raw.SalePrice = np.log(df_raw.SalePrice)

The <code>add_datepart</code> method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [ ]:
add_datepart(df_raw, 'saledate')

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call <code>train_cats</code> to convert strings to pandas categories.

In [ ]:
train_cats(df_raw)

We can specify the order to use for categorical variables if we wish:

In [ ]:
df_raw.UsageBand.cat.categories #return strings list

In [ ]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)

Replace the text categories with numbers, which will make this variable non-categorical

In [ ]:
df_raw.UsageBand = df_raw.UsageBand.cat.codes

To get missing values proportion:

In [ ]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

With <code>proc_df</code> we can replace categories with their numeric codes, handle missing continuous values (replace with the median value, new bolean column to keep track), and split the dependent variable into a separate variables (in this case y becames **Sale Price** column).

In [ ]:
df, y, nas = proc_df(df_raw, 'SalePrice')

### RM model computation

Now is possible to run RM, with **df** (df_raw without independent variable) and **y** as independent variable column.

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

<code>RandomForestRegresson</code> is used for continous values, while <code>RandomForestClassifier</code> is for categorical classification. <code>Fit</code> creates the model, and <code>score</code> return the r squared value.

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

<code>0.9825405552151154</code>

R squared very high could means **overfitting**, to check for it create a validation set

\\( R^{2} \\) tells you how much your model is performing well respect to the naive model that always return the mean

#### How to compute R2

The **total sum of squares** rappresent the proportion of the variance of the data, where \\( y_{i} \\) is the value and \\( \bar{y} \\) is the mean of all values.

\\[ SS_{tot}=\sum (y_{i}-\bar{y})^{2} \\]

The **residual sum of square** reppresent the difference between the data \\( y_{i} \\) and the model result \\( f_{i} \\)

\\[ SS_{res}=\sum (y_{i}-f_{i})^{2} \\]

The \\( R^{2} \\) is calculated as: 

\\[ R^{2} = 1 - \frac{SS_{res}}{SS_{tot}} \\]

The values are always lower than 1 and if \\( R^{2} \\) is lower than 0 means that the model is worst than predicting the mean

### Model validation

The validation set is a smaller than the training and do not contains trainig information

In [ ]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

Then re-compute the model and verify for R2 and RMSE (Root Mean Squared Error, kaggle metric to evaluate the result)

In [ ]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [ ]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

<code>CPU times: user 59.9 s, sys: 390 ms, total: 1min
Wall time: 13.1 s
[0.009070168709057988, 0.025989094636372057, 0.9824145830472109, 0.8775544278864014]</code>

### Subset to speed up the process

We can use <code>subset=</code> to porcess dataframe with <code>proc_df</code>. In order to do not change the validation set you can use <code>_</code> as variable name to do not consider it (python habit). 

In [ ]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=30000)
X_train, _ = split_vals(df_trn, 20000)
y_train, _ = split_vals(y_trn, 20000)

In this way the training process takes way less time but perform worst.

#### Generate a single tree

In [ ]:
m = RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

<code>[17081.175032855852, 17724.36585854255, 0.43350834598758947, 0.46573300556577685]</code>


Where <code>n_estimators</code> is the number of tree and <code>max_depth</code> is the size of the tree, with <code>bootstrap=False</code> we do not randomise the data (**deterministic**).

Then you can draw a tree with:

In [ ]:
draw_tree(m.estimators_[0], df_trn, precision=3)

#### Bagging
Bagging is the strategy to put things togetherin order to sum up the prediction potential and minimalise the error of a single tree. 
In this context you can create a forest with many different tree that has to be as much different as possible at the cost to be little predictive by their own.


The key is to find the ballance between poorly correlated trees done with many small diverse subset and trees higly correlated produced with more or less the same set.

#### Subsampling strategy

Let's say that we have 1M rows dataset and we decide to subsample to 20k, to do that use the command <code>set_rf_sample(20000)</code>. In this way we will have \\(\log_2 20k \\) binary decison and 20k leaf nodes.
Heavely subsampling give low overfitting and smaller trees (faster) but each single tree is less accurate.  
The rule of the thumb is:
- Each estimator (tree) has to be as much accurate as possible
- The correlation between trees has to be as much lower as possible

#### <code>RandomForestRegressor</code> parameters

<code>oob_score=True</code> (default is bootstrap) parameter to get a metric of the error using all the unused rows (kind of validation set) from the original dataset. It tells us if we are overfitting.

<code>min_sample_leaf=2</code> (default is 1) parameter to set the minimum number of element in the leaf node is 2, consequently we will have \\(\log_2 20k-1 \\) binary decision and 10K leaf node.  
Consequence are: 
- more stable average from each tree
- less depth (decisions)
- could avoid overfitting

<code>max_features=0.5</code> is set to use only half (randomly selected)of the columns at each decison. The reason is to increase **variety** of trees not using always the same features, in particular when few trees are produced.

<code>n_jobs=-1</code> is to use all cores available.

#### Features importance

Is very common use logistic regression to quantify feautures imporatnce, in particular the regresson coefficent of each feature is the metric. This is not the best because the result is too dependent on previous assumption and could give missleading results (definitely based more on the strategy than on the data). 

The best way to measure feature importance is to shuffle a column and compare the result. If the there are big changes means **high importance** if the result is the same the feature is not very relevant.  
The code to get a table of it is: 

In [ ]:
fi = rf_feat_importance(m, df_trn)
fi[:10]

Where <code>m</code> is the model and <code>df_trn</code> is the trainig set. The second line of code is used to print the 10 most important featires.

If we have categorical columns like 'High', 'Low' and 'Medium' we can transform them into number 0, 1 and 2 with the function <code>train_cats</code>.  
The other option is to use **one hot encoding** and create 3 new column with just 0/1 values. 

In [ ]:
df_trn2, y_trn, nas = proc_df(df_raw, 'SalePrice', max_n_cat=7)

Where <code>max_n_cat=7</code> mean that every catecorical column is splitted in n different column qit 1/0 values

If there is a particular column in which we want to keep the code, just use <code>train_cats</code> anf then:

In [ ]:
df.COLUMN_NAME = df.COLUMN_NAME.cat.codes

#### Remove redundant features

The fist thing to do is to understand which are the column more similar to each other. Simple correlation between column could not work because it assume **linearity** that is not always present in our dataset.   
The strategy is to use the **ranking correlation** such as **spearmanr** from scipy, but the values couple has to be monotnic (always going up or always down).  
Then you can visualise it with a dendrogram.

In [ ]:
corr = np.round(scipy.stats.spearmanr(df_keep).correlation, 4)
corr_condensed = hc.distance.squareform(1-corr)
z = hc.linkage(corr_condensed, method='average')
fig = plt.figure(figsize=(16,10))
dendrogram = hc.dendrogram(z, labels=df_keep.columns, orientation='left', leaf_font_size=16)
plt.show()

### Partial dependence (what's going on on average)

The standard way to observe dependance is to scatter plot two variables (dependent and one of the independent variable). Sometimes it happens that the points (or better the smoother) show strange behavior due to factor that we are not considering.  
The **partial dependence plot** is a strategy that consists in trying a prediction using our model with the column of interest chagend to a constant, that has to be done for each value or category in order to get the plot (both single prediction and average) and try to interpret our data with less externalities.  
The code to generate the plot is the following:
(always subsample or it will take years)

In [ ]:
from pdpbox import pdp
from plotnine import *

def plot_pdp(feat, clusters=None, feat_name=None):
    feat_name = feat_name or feat
    p = pdp.pdp_isolate(m, x, feat)
    return pdp.pdp_plot(p, feat_name, plot_lines=True,
                        cluster=clusters is not None,
                        n_cluster_centers=clusters)

In [ ]:
plot_pdp('YearMade')

or if we want to see cluster of lines:

In [ ]:
plot_pdp('YearMade', clusters=5)

It is also very useful look at the PD of two independent variables. 

In [ ]:
feats = ['saleElapsed', 'YearMade']
p = pdp.pdp_interact(m, x, feats)
pdp.pdp_interact_plot(p, feats)

### Tree interpreter for single observation analysis

In order to have a look to a single observatio (row) we can use **tree interpreter** packadge.  
We just have to extract the row from our dataset and pass it to <code>ti.predict</code> with the model and then:

In [ ]:
prediction, bias, contributions = ti.predict(m, row)

where <code>prediction</code> is the predicted value, <code>bias</code> is the average value of the dataset and <code>contribution</code> is a metrics about how much a feature is relevant in increasing or decreasing the prediction of the dependent variable. 

The better way to visualise it is the **waterfall chart** where the first "bar" is the average value, the following "bars" are the negative or positive contribution of each feature and the last 2bar2 is the actual final prediction. 

### Training, Validation and Test Sets

In general, the first step is to extract from the dataset random lines to create the validation and the test set. All the remaining rows is the **training** set. To be sure that the result is not working just for the training set, we use the **validation** set to compute the **validation score** and tune all the hyperparameter. Once the we are satisfied we can finally test the model with the **test** set.  
If there is a time component, we can't use randomply picked rows but we have to use the latest data to generate the validation (more recent than training) and the test (more recent than validation) set.  
In this way we are testing if **generalization**  is working in the time dimention, i.e. is our model able to predict the future using the past? 
To check everithing is working you can:
- compute 5 or more model (different parameter or transformation)
- get the validation score
- retrain the model including the validation set
- get the score with the test set
- scatterplot of the resulting score (validation vs test for each model)
- if the we get a linear relationship we are confident in merge validation and training
- if not, rebuilt the models until the relationship is linear between test and validation  

Once you are satisfied with the **validation** set, you can retrain the model **with exactly the same parameters** (here notebook are very helpful) but including also the validation set in the training. You have to be carefull to use the same parameter because at that point you will not be able anymore to "validate" but just use the **test** set.  

#### Validation score vs OOB score

The two scores give a measure of the same thing but the OOB score is sligly less accurate because with the validation set we use the whole forest to make the prediction while the OOB score is the average of the subset of trees containing a given row.

#### Cross-validation

Cross validation is a strategy that consists in randomly shuffle, split the data in 5 chunks and the recursively use one of them as validation set and the remaining four as training set. Each of the five model give a result that can combine with the other one. The drawbacks are: 
- Not working with temporal data
- Slow if big dataset
- The validation set is random

#### Extrapolation

It happens that when you are predicting somthing in the future using data from the past the time component is too strong and and do not allows good prediction.  
In order to quantify the time component importance the steps are:
- create new column telling if the row is or not in the validation set. 
- built a model using the new column as dependent variable
- check the \\( R^{2} \\), if is very high it means that the order (time scale) metter
- check for the features importance and drop the more relevant 
- re-train the model and check again for feature importance, keep track of the more important
- Finally train and score the origina dataset removing one of the **time relevant** column each at time
- check if the score increase or not, if yes remove the column from the dataset because is too much time dependent

### Build your own tree from scratch

To decide where to split a features in the decision tree we have to find the point in which the weighted average of the standard deviation of the two group is as low as possible. 

## Deep learning

Deep learning is useful for model in which we want to explore somthing was not inside the training data, i.e. future time points, unconsidered situation...  

A **neural network** is a linear layer folowed by an activation function, followed by a linear layer and so on..

Differently from RM (care only about the order), we have to normalise the independent variable, because in DP the value matters. 

In [ ]:
mean = x.mean()
std = x.std()

x = (x-mean)/std

In this way we get mean=0 and std=1, **the same mean and std has to be use with the validation set** (results is not perfectly 0 and 1)

In [ ]:
x_valid = (x_valid-mean)/std

If we have a 2D tensor and we want to go back to 3d tesor (the immage) use <code>np.reshape></code>.

In [ ]:
x_imgs = np.reshape(x_valid, (-1,28,28))

Where -1, 28 and 28 mean, all the element (rows) have to be converted in a 3d tensor of 28x28

#### Important

If we use a pretrained mode is mandfatory to use the same normalizzation coefficient used by the author of the model.

### Model with pythorch

In [ ]:
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *

import torch.nn as nn

In [ ]:
net = nn.Sequential(
    nn.Linear(28*28, 10),
    nn.LogSoftmax()
).cuda()

Each input is a vector of size `28*28` pixels and our output is of size `10` (since there are 10 digits: 0, 1, ..., 9). 

We use the output of the final layer to generate our predictions.  Often for classification problems (like MNIST digit classification), the final layer has the same number of outputs as there are classes.  In that case, this is 10: one for each digit from 0 to 9.  These can be converted to comparative probabilities.  For instance, it may be determined that a particular hand-written image is 80% likely to be a 4, 18% likely to be a 9, and 2% likely to be a 3.

Then we have to create a model data `md` object that cointains all our set.

In [ ]:
md = ImageClassifierData.from_arrays('data/path', (x,y), (x_valid, y_valid))

Set the parameter:

In [ ]:
loss=nn.NLLLoss()
metrics=[accuracy]
# opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9)
opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

And then fit the model:

In [ ]:
fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

`n_epochs=5` means go for every image five times.

#### Loss functions and metrics

In machine learning the **loss** function or cost function is representing the price paid for inaccuracy of predictions.

The loss associated with one example in binary classification is given by:
`-(y * log(p) + (1-y) * log (1-p))`
where `y` is the true label of `x` and `p` is the probability predicted by our model that the label is 1.

In [ ]:
def binary_loss(y, p):
    return np.mean(-(y * np.log(p) + (1-y)*np.log(1-p)))

acts = np.array([1, 0, 0, 1])
preds = np.array([0.9, 0.1, 0.2, 0.8])
binary_loss(acts, preds)

Note that in our toy example above our accuracy is 100% and our loss is 0.16

For multi-class classification, we use negative log liklihood (also known as categorical cross entropy) which is exactly the same thing, but summed up over all classes.

#### Test the model

In [ ]:
preds = predict(net, md.val_dl)

The result is a matrix with n rows as the number of element in the validation set and many columns as the `nn.Linear(28*28, `**10**`)` parameter asked for.

In order to explore this matrix use `argmax` that return the index of the highest value of an array. 

In [ ]:
preds.argmax(axis=1)[:5]

This will return the top scoring value (looking at the row, axis=1) for the first 5 observation.  
If we want to store just the best scoring value for each row:

In [ ]:
preds = preds.argmax(1)

Then check how many observation has been properly predicted (**accuracy**):

In [ ]:
np.mean(preds == y_valid)

In this way we have just created **one layer neural net** that is a **logistic regression**. 

#### More layers..

In [ ]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU()
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

One more layer is created adding <code>nn.Linear(100, 10)</code>, this will not change the result because they are just two linear layer (that is equal to one layer).  
In order to make it evolve do **non linear transformation** with <code>nn.ReLU()</code> (Rectified Linear Unit), that rplace all the negatives with **zeros**.

## SUMMARY

In [ ]:
from fastai.imports import *
from fastai.structured import *
from fastai.torch_imports import *
from fastai.io import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *

import torch.nn as nn

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

### RF 

**Pre-pocessing**:
- <code>proc_df</code>
    - split time and date to capture the trend
    - replace nas with column's median (create nas dict)
    - convert string to categories 
    - specify order of levels (low-mid-high)
    - split independent and dependent vabiales
    - one-hot-econded categorical variables (no more than 7-8 leveles)

- <code>split_vals</code>
    - split the dataset in training and validation set 
    - do not shuffle if time is involved

**Compute the model**
- <code>RandomForestRegressor(n_estimators=1, max_depth=3, bootstrap=False, n_jobs=-1)</code>
    - number of tree
    - depth of the tree
**Fit the model**
- <code>m.fit(X_train, y_train)</code>
    
**Predict the dependent variable**
- <code>m.predict(X_train), y_train)</code>

**Check results**
- <code>m.score(X_train, y_train)</code>
    - proportion of the variance in the dependent variable that is predictable from the independent variable
- <code>m.oob_score_</code>
    - quantify overfitting
    - calculate error on the training set, but only include the trees in the calculation of a row's error where that row was not included in training that tree.

**Reduce overfitting**
- Sub-sampling with `set_rf_samples(20000)`
- Increase tree number (bagging) `n_estimators`
- Descrease tree depth `min_samples_leaf`
- Reduce the number of columns used `max_features`

**Tree interpratatio and model improvement**
- Confidence of prediction based on tree variance
- Features importance `rf_feat_importance(m, df_trn)`
- Remove redundant features with hierarchy tree
- Partial dependece plot
- Tree interpreter
- Extrapolation

### NN with SGD 
A neural network is an infinitely flexible function, consisting of layers. A layer is a linear function such as matrix multiplication followed by a non-linear function (the activation).  

The final goal is to find the best paramenter for the a function to approximate aour dataset.  

Most datasets will not be well-represented by a line. We could use a more complicated function, such as 𝑔(𝑥)=𝑎𝑥2+𝑏𝑥+𝑐+sin𝑑. Now we have 4 parameters to learn: 𝑎, 𝑏, 𝑐, and 𝑑. This function is more flexible than 𝑓(𝑥)=𝑎𝑥+𝑏 and will be able to accurately model more datasets.

Neural networks take this to an extreme, and are infinitely flexible. They often have thousands, or even hundreds of thousands of parameters. However the core idea is the same as above. The neural network is a function, and we will learn the best parameters for modeling our data.

#### Pre-processing
- data normalization, `x=(x-mean)/std` to get mean=0 and sd=1
- reshape from 1D to 2D and vice-versa with `np.reshape()`

#### Define the net

- One single layer

In [ ]:
net = nn.Sequential(
    nn.Linear(28*28, 10),
    nn.LogSoftmax()
).cuda()

- Multiple layer

In [ ]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

#### Define the parameter

In [ ]:
loss=nn.NLLLoss()
metrics=[accuracy]
opt=optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In machine learning the **loss** function or cost function is representing the price paid for inaccuracy of predictions.

The loss associated with one example in binary classification is given by:
`-(y * log(p) + (1-y) * log (1-p))`
where `y` is the true label of `x` and `p` is the probability predicted by our model that the label is 1.  

For multi-class classification, we use negative log liklihood (also known as categorical cross entropy) which is exactly the same thing, but summed up over all classes.

#### Fit the model

In [ ]:
fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

#### Make prediction

In [ ]:
preds = predict(net, md.val_dl)